In [1]:
import json
import numpy as np
import pandas
import re
from requests import post
from pandas.io.json import json_normalize

In [2]:
url = 'https://redcap.kpmp.org/api/'
request_body = {
    'token': $TOKEN_HERE,
    'content': 'record',
    'format': 'json',
    'type': 'flat',
    'records[0]': '1-4',
    'records[1]': '1-5',
    'records[2]': '3-2',
    'records[3]': '6-3',
    'records[4]': '14-7',
    'fields[0]': 'study_id',
    'forms[0]': 'demographic_information',
    'events[0]': 'enrollment_arm_1',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'true',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}

In [3]:
response = post(url, data=request_body)
df_1 = json_normalize(response.json())
header_1 = response.headers

In [4]:
df_1 = df_1.drop(columns=['redcap_repeat_instrument','redcap_repeat_instance'])

In [5]:
df_1 = df_1.drop(columns=['demographic_information_complete','date_demographics','income_support_elder','income_support_minor','last_employment_month'])

In [6]:
df_1 = df_1.replace(r'^\s*$', 0, regex=True)

In [7]:
df_1

,study_id,redcap_event_name,gender_salutation___1,gender_salutation___2,gender_salutation___3,gender_salutation___98,gender_salutation_other,relationship_status,education,employment___1,...,income,income_support,insurance___1,insurance___2,insurance___3,insurance___4,insurance___5,insurance___6,insurance___97,insurance___99
0,1-4,enrollment_arm_1,1,0,0,0,0,1,8,0,...,11,2,0,1,0,0,0,0,0,0
1,1-5,enrollment_arm_1,0,1,0,0,0,1,7,1,...,8,1,0,0,0,1,0,0,0,0
2,3-2,enrollment_arm_1,0,1,0,0,0,1,2,0,...,9,3,1,0,0,0,0,0,0,0
3,6-3,enrollment_arm_1,0,1,0,0,0,1,8,0,...,97,0,1,1,0,0,0,0,0,0
4,14-7,enrollment_arm_1,1,0,0,0,0,1,3,1,...,14,2,0,0,0,0,1,0,0,0


In [10]:
def checkbox_column_transformer(data_frame, regex, col_id):
    waitlist = list()
    for col_name in list(data_frame.columns):
        try:
            re.match(regex, col_name).group()
            waitlist.append(col_name)
        except AttributeError:
            continue
            
    for item in waitlist:
        data_frame[item] = data_frame[item].astype(int)
        data_frame[item] = data_frame[item] * int(re.split(regex, item)[1])
    data_frame[f'tmp_{col_id}'] = data_frame[waitlist].values.tolist()
    data_frame = data_frame.drop(columns = waitlist)
    
    checklist = list()
    for entry in data_frame[f'tmp_{col_id}']:
        checklist.append(list(filter(lambda x: x != 0, entry)))
    data_frame = data_frame.drop(columns = f'tmp_{col_id}')
    data_frame[f'{col_id}'] = checklist
    return data_frame

In [11]:
df_1 = checkbox_column_transformer(df_1, regex=r'^gender_salutation___', col_id='gender_salutation')
df_1 = checkbox_column_transformer(df_1, regex=r'^employment___', col_id='employment')
df_1 = checkbox_column_transformer(df_1, regex=r'^insurance___', col_id='insurance')
df_1

,study_id,redcap_event_name,gender_salutation_other,relationship_status,education,last_employment_year,income,income_support,gender_salutation,employment,insurance
0,1-4,enrollment_arm_1,0,1,8,0,11,2,[1],[2],[2]
1,1-5,enrollment_arm_1,0,1,7,0,8,1,[2],[1],[4]
2,3-2,enrollment_arm_1,0,1,2,0,9,3,[2],[2],[1]
3,6-3,enrollment_arm_1,0,1,8,0,97,0,[2],[2],"[1, 2]"
4,14-7,enrollment_arm_1,0,1,3,0,14,2,[1],[1],[5]


In [14]:
request_body = {
    'token': $TOKEN_HERE,
    'content': 'record',
    'format': 'json',
    'type': 'flat',
    'records[0]': '1-4',
    'records[1]': '1-5',
    'records[2]': '3-2',
    'records[3]': '6-3',
    'records[4]': '14-7',
    'fields[0]': 'study_id',
    'fields[1]': 'np_gender',
    'fields[2]': 'np_latino_yn',
    'fields[3]': 'np_race',
    'events[0]': 'screening_arm_1',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'true',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}

In [15]:
response = post(url, data=request_body)
df_2 = json_normalize(response.json())
header_2 = response.headers

In [19]:
df_2 = df_2.drop(columns=['redcap_repeat_instrument','redcap_repeat_instance'])

In [ ]:
df_2 = checkbox_column_transformer(df_2, regex=r'^np_race___', col_id='np_race')

In [ ]:
df_2 = df_2.set_index(['study_id'])
df_2.to_json(orient='index')

In [29]:
request_body = {
    'token': $TOKEN_HERE,
    'content': 'record',
    'format': 'json',
    'type': 'flat',
    'records[0]': '1-4',
    'records[1]': '1-5',
    'records[2]': '3-2',
    'records[3]': '6-3',
    'records[4]': '14-7',
    'fields[0]': 'study_id',
    'forms[0]': 'coordinator_medical_history',
    'events[0]': 'enrollment_arm_1',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'true',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}

In [30]:
response = post(url, data=request_body)
df_3 = json_normalize(response.json())
header_3 = response.headers

In [31]:
df_3 = df_3.drop(columns=['redcap_repeat_instrument','redcap_repeat_instance'])
df_3 = df_3.drop(columns=['cmh_date'])

'{"1-4":{"redcap_event_name":"screening_arm_1","np_gender":"1","np_latino_yn":"0","np_race":[5]},"1-5":{"redcap_event_name":"screening_arm_1","np_gender":"2","np_latino_yn":"0","np_race":[3]},"3-2":{"redcap_event_name":"screening_arm_1","np_gender":"2","np_latino_yn":"0","np_race":[3]},"6-3":{"redcap_event_name":"screening_arm_1","np_gender":"2","np_latino_yn":"0","np_race":[5]},"14-7":{"redcap_event_name":"screening_arm_1","np_gender":"1","np_latino_yn":"0","np_race":[5]}}'